In [ ]:
import pandas as pd
import re

# Load raw data
df = pd.read_excel('fatsecret_raw_data_dump.xlsx')

df.head()

,food_id,food_name,food_name_id,food_type,description,search_keyword
0,3092,Egg,Telur,Generic,Per 100g - Calories: 147kcal | Fat: 9.94g | Ca...,Egg
1,33797,Fried Egg,Telur Goreng,Generic,Per 100g - Calories: 201kcal | Fat: 15.31g | C...,Egg
2,3094,Boiled Egg,Telur Rebus,Generic,Per 100g - Calories: 155kcal | Fat: 10.61g | C...,Egg
3,33793,Egg White,Putih Telur,Generic,Per 100g - Calories: 52kcal | Fat: 0.17g | Car...,Egg
4,39866,Scrambled Egg,Telur Orak-arik,Generic,Per 100g - Calories: 212kcal | Fat: 16.18g | C...,Egg


# Parse Nutrition from Description

In [ ]:
# Function to parse nutrition from description
def parse_nutrition(description):
    """
    Extract nutrition values from FatSecret description string

    Example input: "Per 100g - Calories: 147kcal | Fat: 9.94g | Carbs: 0.77g | Protein: 12.58g"
    Returns: dict with serving_size, serving_unit, calories, protein, carbs, fat
    """

    # Initialize default values (in case parsing fails)
    result = {
        'serving_size': None,
        'serving_unit': None,
        'calories': None,
        'protein': None,
        'carbs': None,
        'fat': None
    }

    if pd.isna(description):
        return result

    # Extract serving size and unit
    # Pattern: "Per [number][unit]" e.g., "Per 100g" or "Per 168ml"
    serving_match = re.search(r'Per\s+(\d+\.?\d*)\s*([a-zA-Z]+)', description)
    if serving_match:
        result['serving_size'] = float(serving_match.group(1))
        result['serving_unit'] = serving_match.group(2).lower()  # 'g', 'ml', etc.

    # Extract calories
    # Pattern: "Calories: [number]kcal"
    cal_match = re.search(r'Calories:\s*(\d+\.?\d*)\s*kcal', description)
    if cal_match:
        result['calories'] = float(cal_match.group(1))

    # Extract fat
    # Pattern: "Fat: [number]g"
    fat_match = re.search(r'Fat:\s*(\d+\.?\d*)\s*g', description)
    if fat_match:
        result['fat'] = float(fat_match.group(1))

    # Extract carbs
    # Pattern: "Carbs: [number]g"
    carbs_match = re.search(r'Carbs:\s*(\d+\.?\d*)\s*g', description)
    if carbs_match:
        result['carbs'] = float(carbs_match.group(1))

    # Extract protein
    # Pattern: "Protein: [number]g"
    protein_match = re.search(r'Protein:\s*(\d+\.?\d*)\s*g', description)
    if protein_match:
        result['protein'] = float(protein_match.group(1))

    return result

# Apply parsing function to all rows
print("Parsing nutrition values...")
nutrition_data = df['description'].apply(parse_nutrition)

# Convert list of dicts to DataFrame and join with original
nutrition_df = pd.DataFrame(nutrition_data.tolist())
df = pd.concat([df, nutrition_df], axis=1)

Parsing nutrition values...


In [ ]:
# Check for parsing failures (null values)
null_counts = df[['serving_size', 'serving_unit', 'calories', 'protein', 'carbs', 'fat']].isnull().sum()
print("Missing values after parsing:")

print(null_counts)

if null_counts.sum() > 0:
    print(f"{null_counts.sum()} total missing values found")
    print("Sample rows with missing data:")
    print(df[df[['serving_size', 'calories', 'protein']].isnull().any(axis=1)][['food_name', 'description']].head(3))
else:
    print("No missing values - all descriptions parsed successfully!")


Missing values after parsing:
serving_size    0
serving_unit    0
calories        0
protein         0
carbs           0
fat             0
dtype: int64
No missing values - all descriptions parsed successfully!


In [ ]:
# Check value ranges (sanity checks)
print("Nutrition value ranges:")
print(df[['serving_size', 'calories', 'protein', 'carbs', 'fat']].describe().round(2))

Nutrition value ranges:
       serving_size  calories  protein   carbs     fat
count        233.00    233.00   233.00  233.00  233.00
mean         229.20    337.81    22.47   25.54   16.34
std          388.44    525.97    42.55   46.97   28.59
min           12.00      1.00     0.00    0.00    0.00
25%          100.00     65.00     2.43    2.08    0.83
50%          100.00    173.00     8.80    9.40    4.59
75%          162.00    312.00    22.09   26.69   17.39
max         3887.00   2707.00   230.36  353.32  157.86


In [ ]:
# Check serving units distribution
print("Serving unit distribution:")
print(df['serving_unit'].value_counts())

Serving unit distribution:
serving_unit
g    233
Name: count, dtype: int64


In [ ]:
# Sample output
print("\n Sample parsed data (first 5 rows):")
print(df[['food_name', 'serving_size', 'serving_unit', 'calories', 'protein', 'carbs', 'fat']].head())


 Sample parsed data (first 5 rows):
       food_name  serving_size serving_unit  calories  protein  carbs    fat
0            Egg         100.0            g     147.0    12.58   0.77   9.94
1      Fried Egg         100.0            g     201.0    13.63   0.88  15.31
2     Boiled Egg         100.0            g     155.0    12.58   1.12  10.61
3      Egg White         100.0            g      52.0    10.90   0.73   0.17
4  Scrambled Egg         100.0            g     212.0    13.84   2.08  16.18


In [ ]:
df.shape

(233, 12)

# Deduplication & Data Cleaning

In [ ]:
# Check how many entries are per 100g vs other serving sizes
print("Serving size distribution:")
print(df['serving_size'].value_counts().head(10))

serving_100g = len(df[df['serving_size'] == 100.0])
serving_other = len(df[df['serving_size'] != 100.0])

print(f"\nPer 100g: {serving_100g} rows ({serving_100g/len(df)*100:.1f}%)")
print(f"Other sizes: {serving_other} rows ({serving_other/len(df)*100:.1f}%)")

Serving size distribution:
serving_size
100.0     125
101.0      22
1086.0      3
973.0       3
162.0       3
168.0       3
157.0       2
133.0       2
181.0       2
175.0       2
Name: count, dtype: int64

Per 100g: 125 rows (53.6%)
Other sizes: 108 rows (46.4%)


## STRATEGY 1: Prefer 100g entries

In [ ]:
# Group by food_name, check if 100g entry exists
df['has_100g_entry'] = df.groupby('food_name')['serving_size'].transform(lambda x: (x == 100.0).any())

# For foods WITH 100g entry: keep only the 100g one
# For foods WITHOUT 100g entry: keep all (will standardize later)
df_cleaned = df[
    ((df['has_100g_entry'] == True) & (df['serving_size'] == 100.0)) |  # Has 100g → keep only 100g
    (df['has_100g_entry'] == False)                                      # No 100g → keep all
].copy()

print(f"After preferring 100g entries: {len(df_cleaned)} rows")
print(f"Removed {len(df) - len(df_cleaned)} duplicate serving sizes")

After preferring 100g entries: 233 rows
Removed 0 duplicate serving sizes


## STRATEGY 2: Remove exact duplicates

In [ ]:
# Check for exact duplicate food_names (same name, same nutrition)
duplicates = df_cleaned[df_cleaned.duplicated(subset=['food_name', 'calories', 'protein', 'carbs', 'fat'], keep='first')]

if len(duplicates) > 0:
    print(f"\nFound {len(duplicates)} exact duplicates (same name + nutrition):")
    print(duplicates[['food_name', 'serving_size', 'calories', 'protein']].head())

    # Remove exact duplicates
    df_cleaned = df_cleaned.drop_duplicates(subset=['food_name', 'calories', 'protein', 'carbs', 'fat'], keep='first')
    print(f"\nRemoved exact duplicates: {len(df_cleaned)} rows remaining")
else:
    print("\nNo exact duplicates found")


No exact duplicates found


## STRATEGY 3: Identify similar names (for awareness)

In [ ]:
# Find foods with very similar names (might be duplicates)
print("\nFoods with similar names (will filter manually in Step 12):")

# Group by search_keyword to see similar foods
keyword_counts = df_cleaned['search_keyword'].value_counts()
redundant_keywords = keyword_counts[keyword_counts > 5].head()

if len(redundant_keywords) > 0:
    print("\nKeywords with >5 entries (might have redundancy):")
    print(redundant_keywords)

    # Example: Show first redundant category
    example_keyword = redundant_keywords.index[0]
    print(f"\nExample: '{example_keyword}' category foods:")
    example_foods = df_cleaned[df_cleaned['search_keyword'] == example_keyword][
        ['food_name', 'serving_size', 'calories', 'protein']
    ].head(10)
    print(example_foods.round(2))
    print(f"\nNote: Will select best {example_keyword} variants in Step 12 (final 150 selection)")

# Drop temporary column
df_cleaned = df_cleaned.drop(columns=['has_100g_entry'])


Foods with similar names (will filter manually in Step 12):

Keywords with >5 entries (might have redundancy):
search_keyword
Coffee      11
Chicken     11
Tea         10
Beef         9
Mackerel     8
Name: count, dtype: int64

Example: 'Coffee' category foods:
                       food_name  serving_size  calories  protein
5   Coffee (Brewed From Grounds)         100.0       1.0     0.12
6              Coffee with Cream         168.0      25.0     0.37
7    Coffee with Cream and Sugar         175.0      49.0     0.37
8               Coffee with Milk         168.0       6.0     0.31
9     Coffee with Milk and Sugar         175.0      30.0     0.31
10             Coffee with Sugar         171.0      28.0     0.18
11                  Latte Coffee         332.0     110.0     7.17
12                    Cappuccino         181.0      56.0     3.07
13         Coffee with Skim Milk         168.0       5.0     0.32
14          Decaffeinated Coffee         100.0       1.0     0.10

Note: Will

In [ ]:
print("DEDUPLICATION SUMMARY")
print(f"Original rows: {len(df)}")
print(f"After deduplication: {len(df_cleaned)}")
print(f"Removed: {len(df) - len(df_cleaned)} rows ({(len(df) - len(df_cleaned))/len(df)*100:.1f}%)")

print(f"\nServing size distribution after dedup:")
print(df_cleaned['serving_size'].value_counts().head(10))

print("\nSample cleaned data:")
print(df_cleaned[['food_name', 'serving_size', 'serving_unit', 'calories', 'protein']].head(10).round(2))

# Update df to cleaned version
df = df_cleaned.copy()

print(f"STEP 2 COMPLETE")
print(f"Shape: {df.shape}")
print(f"Cleaned dataset ready for Step 3 (Standardization)!")

DEDUPLICATION SUMMARY
Original rows: 233
After deduplication: 233
Removed: 0 rows (0.0%)

Serving size distribution after dedup:
serving_size
100.0     125
101.0      22
1086.0      3
973.0       3
162.0       3
168.0       3
157.0       2
133.0       2
181.0       2
175.0       2
Name: count, dtype: int64

Sample cleaned data:
                      food_name  serving_size serving_unit  calories  protein
0                           Egg         100.0            g     147.0    12.58
1                     Fried Egg         100.0            g     201.0    13.63
2                    Boiled Egg         100.0            g     155.0    12.58
3                     Egg White         100.0            g      52.0    10.90
4                 Scrambled Egg         100.0            g     212.0    13.84
5  Coffee (Brewed From Grounds)         100.0            g       1.0     0.12
6             Coffee with Cream         168.0            g      25.0     0.37
7   Coffee with Cream and Sugar         175.0 

# Standardize Nutrition per 100g

* Normalize all nutrition values to per 100g basis
* Essential for fair comparison across foods

In [ ]:
print(f"\nCurrent shape: {df.shape}")

# Check current serving size distribution
print("\nCurrent serving sizes:")
print(df['serving_size'].value_counts().head())

non_100g = len(df[df['serving_size'] != 100.0])
print(f"\nRows needing standardization: {non_100g} ({non_100g/len(df)*100:.1f}%)")



Current shape: (233, 12)

Current serving sizes:
serving_size
100.0     125
101.0      22
1086.0      3
973.0       3
162.0       3
Name: count, dtype: int64

Rows needing standardization: 108 (46.4%)


In [ ]:
print("\nCalculating per 100g values...")

# Calculate scale factor: 100 / serving_size
df['scale_factor'] = 100.0 / df['serving_size']

# Apply scaling to all nutrition values
df['calories_per_100g'] = df['calories'] * df['scale_factor']
df['protein_per_100g'] = df['protein'] * df['scale_factor']
df['carbs_per_100g'] = df['carbs'] * df['scale_factor']
df['fat_per_100g'] = df['fat'] * df['scale_factor']

# Drop temporary scale_factor column
df = df.drop(columns=['scale_factor'])

print("Standardization complete!")


Calculating per 100g values...
Standardization complete!


In [ ]:
# Check for invalid values (NaN, infinity, negative)
invalid_calories = df[df['calories_per_100g'].isna() | (df['calories_per_100g'] < 0) | (df['calories_per_100g'] > 900)]
invalid_protein = df[df['protein_per_100g'].isna() | (df['protein_per_100g'] < 0) | (df['protein_per_100g'] > 100)]
invalid_fat = df[df['fat_per_100g'].isna() | (df['fat_per_100g'] < 0) | (df['fat_per_100g'] > 100)]
invalid_carbs = df[df['carbs_per_100g'].isna() | (df['carbs_per_100g'] < 0) | (df['carbs_per_100g'] > 100)]

print("\nInvalid values check:")
print(f"  Invalid calories: {len(invalid_calories)} rows")
print(f"  Invalid protein: {len(invalid_protein)} rows")
print(f"  Invalid fat: {len(invalid_fat)} rows")
print(f"  Invalid carbs: {len(invalid_carbs)} rows")

if len(invalid_calories) > 0 or len(invalid_protein) > 0:
    print("\nSample invalid rows:")
    invalid_all = pd.concat([invalid_calories, invalid_protein, invalid_fat, invalid_carbs]).drop_duplicates()
    print(invalid_all[['food_name', 'serving_size', 'calories_per_100g', 'protein_per_100g']].head())


Invalid values check:
  Invalid calories: 0 rows
  Invalid protein: 0 rows
  Invalid fat: 0 rows
  Invalid carbs: 0 rows


In [ ]:
# Check value ranges
print("\nNutrition value ranges (per 100g):")
stats_cols = ['calories_per_100g', 'protein_per_100g', 'fat_per_100g', 'carbs_per_100g']
print(df[stats_cols].describe().round(2))


Nutrition value ranges (per 100g):
       calories_per_100g  protein_per_100g  fat_per_100g  carbs_per_100g
count             233.00            233.00        233.00          233.00
mean              160.66             10.05          8.20           12.27
std               133.30              9.45         10.80           14.64
min                 0.67              0.00          0.00            0.00
25%                53.70              1.92          0.54            1.24
50%               136.63              6.29          3.32            7.04
75%               230.00             18.19         11.45           19.07
max               597.00             39.58         52.83           77.78


In [ ]:
# Sanity checks
print("\nSanity checks:")
print(f"  Max calories per 100g: {df['calories_per_100g'].max():.2f} (should be < 900)")
print(f"  Max protein per 100g: {df['protein_per_100g'].max():.2f} (should be < 100)")
print(f"  Max fat per 100g: {df['fat_per_100g'].max():.2f} (should be < 100)")
print(f"  Max carbs per 100g: {df['carbs_per_100g'].max():.2f} (should be < 100)")


Sanity checks:
  Max calories per 100g: 597.00 (should be < 900)
  Max protein per 100g: 39.58 (should be < 100)
  Max fat per 100g: 52.83 (should be < 100)
  Max carbs per 100g: 77.78 (should be < 100)


In [ ]:
# Check macronutrient sum (rough check: protein + carbs + fat should be reasonable)
df['macro_sum'] = df['protein_per_100g'] + df['carbs_per_100g'] + df['fat_per_100g']
unusual_macro = df[df['macro_sum'] > 100]
print(f"\nFoods with macro sum > 100g: {len(unusual_macro)} (might be data errors)")
if len(unusual_macro) > 0:
    print(unusual_macro[['food_name', 'protein_per_100g', 'carbs_per_100g', 'fat_per_100g', 'macro_sum']].head())

df = df.drop(columns=['macro_sum'])


Foods with macro sum > 100g: 0 (might be data errors)


In [ ]:
# Compare original vs standardized (verification)
print("\nComparison: Original vs Standardized (sample):")
comparison_cols = ['food_name', 'serving_size', 'calories', 'calories_per_100g', 'protein', 'protein_per_100g']
print(df[comparison_cols].head(10).round(2))


Comparison: Original vs Standardized (sample):
                      food_name  serving_size  calories  calories_per_100g  \
0                           Egg         100.0     147.0             147.00   
1                     Fried Egg         100.0     201.0             201.00   
2                    Boiled Egg         100.0     155.0             155.00   
3                     Egg White         100.0      52.0              52.00   
4                 Scrambled Egg         100.0     212.0             212.00   
5  Coffee (Brewed From Grounds)         100.0       1.0               1.00   
6             Coffee with Cream         168.0      25.0              14.88   
7   Coffee with Cream and Sugar         175.0      49.0              28.00   
8              Coffee with Milk         168.0       6.0               3.57   
9    Coffee with Milk and Sugar         175.0      30.0              17.14   

   protein  protein_per_100g  
0    12.58             12.58  
1    13.63             13.63  


# Extract Category from Keyword

In [ ]:
# Define category keywords
protein_keywords = ['egg', 'chicken', 'beef', 'fish', 'tuna', 'salmon',
                    'shrimp', 'squid', 'crab', 'lobster', 'mackerel', 'catfish',
                    'anchovy', 'sardines', 'tofu', 'tempeh', 'meatballs', 'chicken sausage',
                    'beef sausage', 'soybeans']
carbs_keywords = ['rice', 'brown rice', 'white rice', 'fried rice',
                  'noodles', 'rice noodles', 'spaghetti', 'bread',
                  'potatoes', 'sweet potato', 'oatmeal', 'cereal',
                  'biscuits', 'popcorn']
veg_keywords = ['spinach', 'broccoli', 'mushrooms', 'green beans',
                'peas', 'cauliflower', 'squash', 'eggplant',
                'long beans', 'black beans', 'red beans', 'pickles']
fruit_keywords = ['apple', 'banana', 'orange', 'mango', 'pineapple',
                  'grapes', 'strawberries', 'blueberries', 'pears',
                  'avocado', 'tangerine', 'coconut']
beverage_keywords = ['coffee', 'tea', 'juice', 'milk']
dairy_keywords = ['cheese', 'yogurt']
nut_keywords = ['almonds', 'cashews', 'peanut', 'walnut']
mixed_keywords = ['soup', 'salad', 'curry', 'sandwich', 'nuggets']
dessert_keywords = ['ice cream', 'pudding']

def categorize_food(row):
    """
    Categorize food based on keyword matching + nutrition profile
    Returns category string
    """
    keyword = str(row['search_keyword']).lower()
    food_name = str(row['food_name']).lower()

    # Combine for matching
    text = keyword + ' ' + food_name

    # Keyword-based matching
    if any(kw in text for kw in protein_keywords):
        return 'protein'
    elif any(kw in text for kw in carbs_keywords):
        return 'carbs'
    elif any(kw in text for kw in veg_keywords):
        return 'vegetables'
    elif any(kw in text for kw in fruit_keywords):
        return 'fruits'
    elif any(kw in text for kw in beverage_keywords):
        return 'beverages'
    elif any(kw in text for kw in dairy_keywords):
        return 'dairy'
    elif any(kw in text for kw in nut_keywords):
        return 'nuts'
    elif any(kw in text for kw in mixed_keywords):
        return 'mixed'
    elif any(kw in text for kw in dessert_keywords):
        return 'desserts'

    # Fallback: Nutrition-based classification
    protein_ratio = row['protein_per_100g'] / max(row['calories_per_100g'], 1)
    carb_ratio = row['carbs_per_100g'] / max(row['calories_per_100g'], 1)
    fat_ratio = row['fat_per_100g'] / max(row['calories_per_100g'], 1)

    # High protein (>0.15 protein/cal ratio)
    if protein_ratio > 0.15:
        return 'protein'
    # High carb (>0.20 carb/cal ratio)
    elif carb_ratio > 0.20:
        return 'carbs'
    # High fat (>0.10 fat/cal ratio) + low cal (likely nuts)
    elif fat_ratio > 0.10 and row['calories_per_100g'] > 400:
        return 'nuts'
    # Very low cal (likely vegetables or beverages)
    elif row['calories_per_100g'] < 50:
        if carb_ratio > 0.05:
            return 'vegetables'
        else:
            return 'beverages'

    # Default
    return 'other'

df['category'] = df.apply(categorize_food, axis=1)

In [ ]:
# Count desserts before dropping
desserts_count = len(df[df['category'] == 'desserts'])
print(f"\nDesserts identified: {desserts_count} items")

if desserts_count > 0:
    print("\nDesserts to be dropped:")
    print(df[df['category'] == 'desserts'][['food_name', 'food_name_id', 'search_keyword']].head(10))

# DROP DESSERTS
df = df[df['category'] != 'desserts'].copy()

print(f"\nFiltered out {desserts_count} dessert items")
print(f"Remaining foods: {len(df)}")


Desserts identified: 6 items

Desserts to be dropped:
                                   food_name                   food_name_id  \
32                                 Ice Cream                        Es Krim   
33                        Vanilla Ice Creams                 Es Krim Vanila   
34  Ice Cream (Flavors Other Than Chocolate)  Es Krim (Rasa Selain Cokelat)   
35                       Chocolate Ice Cream                Es Krim Cokelat   
36                                   Pudding                         Puding   
37                          Vanilla Puddings                  Puding Vanila   

   search_keyword  
32      Ice Cream  
33      Ice Cream  
34      Ice Cream  
35      Ice Cream  
36        Pudding  
37        Pudding  

Filtered out 6 dessert items
Remaining foods: 227


In [ ]:
# Check for uncategorized ('other')
uncategorized = df[df['category'] == 'other']
print(f"\nUncategorized foods: {len(uncategorized)} rows ({len(uncategorized)/len(df)*100:.1f}%)")

if len(uncategorized) > 0:
    print("\nFoods marked as 'other' (may need manual review):")
    print(uncategorized[['food_name', 'food_name_id', 'search_keyword', 'calories_per_100g', 'protein_per_100g']].head(10))


Uncategorized foods: 0 rows (0.0%)


In [ ]:
# Category distribution
print("\nCategory distribution:")
category_counts = df['category'].value_counts()
print(category_counts)


Category distribution:
category
protein       94
carbs         38
beverages     27
vegetables    25
fruits        17
dairy         11
mixed          8
nuts           7
Name: count, dtype: int64


In [ ]:
print("\nCategory percentages:")
category_pct = (df['category'].value_counts() / len(df) * 100).round(1)
for cat, pct in category_pct.items():
    print(f"  {cat}: {pct}%")


Category percentages:
  protein: 41.4%
  carbs: 16.7%
  beverages: 11.9%
  vegetables: 11.0%
  fruits: 7.5%
  dairy: 4.8%
  mixed: 3.5%
  nuts: 3.1%


In [ ]:
# Sample foods per category
print("\nSample foods per category (first 3 each):")
for category in sorted(df['category'].unique()):
    if category != 'other':
        sample = df[df['category'] == category][['food_name', 'food_name_id']].head(3)
        print(f"\n{category.upper()}:")
        for idx, row in sample.iterrows():
            print(f"  - {row['food_name']} / {row['food_name_id']}")


Sample foods per category (first 3 each):

BEVERAGES:
  - Coffee (Brewed From Grounds) / Kopi (Diseduh dari Bubuk)
  - Coffee with Cream / Kopi dengan Krim
  - Coffee with Cream and Sugar / Kopi dengan Krim dan Gula

CARBS:
  - Roasted Potato / Kentang Panggang
  - Potato / Kentang
  - Boiled Potato / Kentang Rebus

DAIRY:
  - Ham and Cheese Sandwich / Sandwich Ham dan Keju
  - Ham and Cheese Sandwich with Lettuce and Spread / Sandwich Ham dan Keju dengan Selada dan Saus
  - Cheese Sandwich / Sandwich Keju

FRUITS:
  - Avocado / Alpukat
  - Grapes (Red or Green, European Type Varieties Such As Thompson Seedless) / Anggur (Merah atau Hijau, Varietas Eropa seperti Thompson Seedless)
  - Grapes / Anggur

MIXED:
  - Vegetable Soup (Home Recipe) / Sup Sayur (Resep Rumahan)
  - Tomato Soup / Sup Tomat
  - Mixed Beans Soup / Sup Kacang Campur

NUTS:
  - Almonds / Almond
  - Dry Roasted Almonds (Without Salt Added) / Almond Panggang Kering (Tanpa Garam Tambahan)
  - Dry Roasted Almonds (with 

In [ ]:
print(f"Shape: {df.shape}")
print(f"New column added: category")
print(f"Categorized: {len(df[df['category'] != 'other'])}/{len(df)} foods")

Shape: (227, 17)
New column added: category
Categorized: 227/227 foods


In [ ]:
# Count beverages before dropping
beverages = df[df['category'] == 'beverages']
beverages_count = len(beverages)

print(f"\nBeverages identified: {beverages_count} items ({beverages_count/len(df)*100:.1f}%)")

if beverages_count > 0:
    print("\nBeverages to be dropped:")
    print(beverages[['food_name', 'food_name_id', 'calories_per_100g']].head(15))
    print(f"... and {max(0, beverages_count - 15)} more")

# DROP BEVERAGES
df = df[df['category'] != 'beverages'].copy()

print(f"\nFiltered out {beverages_count} beverage items")
print(f"\n Remaining foods: {len(df)}")



Beverages identified: 27 items (11.9%)

Beverages to be dropped:
                       food_name               food_name_id  calories_per_100g
5   Coffee (Brewed From Grounds)  Kopi (Diseduh dari Bubuk)           1.000000
6              Coffee with Cream           Kopi dengan Krim          14.880952
7    Coffee with Cream and Sugar  Kopi dengan Krim dan Gula          28.000000
8               Coffee with Milk           Kopi dengan Susu           3.571429
9     Coffee with Milk and Sugar  Kopi dengan Susu dan Gula          17.142857
10             Coffee with Sugar           Kopi dengan Gula          16.374269
11                  Latte Coffee                 Kopi Latte          33.132530
12                    Cappuccino                 Cappuccino          30.939227
13         Coffee with Skim Milk      Kopi dengan Susu Skim           2.976190
14          Decaffeinated Coffee          Kopi Tanpa Kafein           1.000000
15               Espresso Coffee              Kopi Espresso      

In [ ]:
# Check category distribution after drop
print("\nCategory distribution after dropping beverages:")
category_counts = df['category'].value_counts()
print(category_counts)

print("\nCategory percentages:")
category_pct = (df['category'].value_counts() / len(df) * 100).round(1)
for cat, pct in category_pct.items():
    print(f"  {cat}: {pct}%")



Category distribution after dropping beverages:
category
protein       94
carbs         38
vegetables    25
fruits        17
dairy         11
mixed          8
nuts           7
Name: count, dtype: int64

Category percentages:
  protein: 47.0%
  carbs: 19.0%
  vegetables: 12.5%
  fruits: 8.5%
  dairy: 5.5%
  mixed: 4.0%
  nuts: 3.5%


# Meal Suitability Flags

Assign suitability flags for each meal type based on category and nutrition profile

In [ ]:
def assign_meal_suitability(row):
    """
    Assign meal suitability with Indonesian meal culture context
    More specific rules to avoid overlap
    """
    category = row['category']
    food_name = str(row['food_name']).lower()
    calories = row['calories_per_100g']

    # Initialize
    breakfast = 0
    lunch = 0
    dinner = 0
    snack = 0

    # ==========================================
    # BREAKFAST (Sarapan)
    # Light, quick items: eggs, bread, coffee, fruits
    # ==========================================
    if category == 'beverages':
        breakfast = 1
        snack = 1  # Coffee/tea also good for snacks

    elif category == 'fruits':
        breakfast = 1
        snack = 1  # Fruits for both

    elif category == 'dairy':
        breakfast = 1
        snack = 1  # Yogurt for both

    elif category == 'protein':
        # ONLY eggs and tofu/tempeh for breakfast
        if any(kw in food_name for kw in ['egg']):
            breakfast = 1
        elif any(kw in food_name for kw in ['tofu', 'tempeh']):
            breakfast = 1
            lunch = 1  # Also OK for lunch/dinner
            dinner = 1
        else:
            # Other proteins (chicken, beef, fish) NOT for breakfast
            pass

    elif category == 'carbs':
        if any(kw in food_name for kw in ['bread', 'oatmeal', 'cereal']):
            breakfast = 1
            snack = 1  # Bread/biscuits also snacks
        elif any(kw in food_name for kw in ['fried rice']):
            breakfast = 1  # Nasi goreng = Indonesian breakfast!
            lunch = 1
            dinner = 1

    # ==========================================
    # LUNCH (Makan Siang)
    # Substantial meals: rice, protein, vegetables
    # ==========================================
    if category == 'carbs':
        # Rice, noodles for lunch/dinner (not breakfast bread)
        if any(kw in food_name for kw in ['rice', 'noodle', 'spaghetti', 'potato']):
            lunch = 1
            dinner = 1

    elif category == 'protein':
        # Main proteins for lunch/dinner (NOT eggs)
        if not any(kw in food_name for kw in ['egg']):
            lunch = 1
            dinner = 1

    elif category == 'vegetables':
        lunch = 1
        dinner = 1

    elif category == 'mixed':
        # Soups, salads, curries
        lunch = 1
        dinner = 1

    # ==========================================
    # SNACK (Camilan)
    # Small, portable items
    # ==========================================
    if category == 'nuts':
        snack = 1

    elif category == 'carbs':
        if any(kw in food_name for kw in ['popcorn', 'crackers', 'biscuit']):
            snack = 1

    return breakfast, lunch, dinner, snack

# Apply function and create new columns
df[['suitable_breakfast', 'suitable_lunch', 'suitable_dinner', 'suitable_snack']] = \
    df.apply(assign_meal_suitability, axis=1, result_type='expand')

In [ ]:
# Check if every food is suitable for at least 1 meal
no_suitability = df[
    (df['suitable_breakfast'] == 0) &
    (df['suitable_lunch'] == 0) &
    (df['suitable_dinner'] == 0) &
    (df['suitable_snack'] == 0)
]

print(f"\nFoods with NO meal suitability: {len(no_suitability)} ({len(no_suitability)/len(df)*100:.1f}%)")

if len(no_suitability) > 0:
    print("\nWarning: These foods are not suitable for any meal type:")
    print(no_suitability[['food_name', 'food_name_id', 'category', 'calories_per_100g']].head(10))
    print("\nConsider reviewing these manually or adding suitability rules")


Foods with NO meal suitability: 3 (1.5%)

         food_name food_name_id category  calories_per_100g
68   Jam Preserves   Selai Buah    carbs              260.0
114       Baguette     Baguette    carbs              274.0
116    Raisin Bran  Raisin Bran    carbs              321.0

Consider reviewing these manually or adding suitability rules


In [ ]:
# Distribution by meal type
print("\nMeal type distribution:")
print(f"  Suitable for breakfast: {df['suitable_breakfast'].sum()} foods ({df['suitable_breakfast'].sum()/len(df)*100:.1f}%)")
print(f"  Suitable for lunch: {df['suitable_lunch'].sum()} foods ({df['suitable_lunch'].sum()/len(df)*100:.1f}%)")
print(f"  Suitable for dinner: {df['suitable_dinner'].sum()} foods ({df['suitable_dinner'].sum()/len(df)*100:.1f}%)")
print(f"  Suitable for snack: {df['suitable_snack'].sum()} foods ({df['suitable_snack'].sum()/len(df)*100:.1f}%)")


Meal type distribution:
  Suitable for breakfast: 53 foods (26.5%)
  Suitable for lunch: 138 foods (69.0%)
  Suitable for dinner: 138 foods (69.0%)
  Suitable for snack: 49 foods (24.5%)


In [ ]:
# Check balance across categories
print("\nCategory breakdown by meal type:")
print("\nBREAKFAST foods by category:")
print(df[df['suitable_breakfast'] == 1]['category'].value_counts())

print("\nLUNCH foods by category:")
print(df[df['suitable_lunch'] == 1]['category'].value_counts())

print("\nDINNER foods by category:")
print(df[df['suitable_dinner'] == 1]['category'].value_counts())

print("\nSNACK foods by category:")
print(df[df['suitable_snack'] == 1]['category'].value_counts())


Category breakdown by meal type:

BREAKFAST foods by category:
category
fruits     17
protein    15
dairy      11
carbs      10
Name: count, dtype: int64

LUNCH foods by category:
category
protein       84
vegetables    25
carbs         21
mixed          8
Name: count, dtype: int64

DINNER foods by category:
category
protein       84
vegetables    25
carbs         21
mixed          8
Name: count, dtype: int64

SNACK foods by category:
category
fruits    17
carbs     14
dairy     11
nuts       7
Name: count, dtype: int64


In [ ]:
# Sample foods per meal type
print("\nSample suitable foods (first 5 each):")

print("\nBREAKFAST:")
breakfast_sample = df[df['suitable_breakfast'] == 1][['food_name', 'food_name_id', 'category']].head(5)
for idx, row in breakfast_sample.iterrows():
    print(f"  - {row['food_name']} / {row['food_name_id']} ({row['category']})")

print("\nLUNCH:")
lunch_sample = df[df['suitable_lunch'] == 1][['food_name', 'food_name_id', 'category']].head(5)
for idx, row in lunch_sample.iterrows():
    print(f"  - {row['food_name']} / {row['food_name_id']} ({row['category']})")

print("\nDINNER:")
dinner_sample = df[df['suitable_dinner'] == 1][['food_name', 'food_name_id', 'category']].head(5)
for idx, row in dinner_sample.iterrows():
    print(f"  - {row['food_name']} / {row['food_name_id']} ({row['category']})")

print("\nSNACK:")
snack_sample = df[df['suitable_snack'] == 1][['food_name', 'food_name_id', 'category']].head(5)
for idx, row in snack_sample.iterrows():
    print(f"  - {row['food_name']} / {row['food_name_id']} ({row['category']})")


Sample suitable foods (first 5 each):

BREAKFAST:
  - Egg / Telur (protein)
  - Fried Egg / Telur Goreng (protein)
  - Boiled Egg / Telur Rebus (protein)
  - Egg White / Putih Telur (protein)
  - Scrambled Egg / Telur Orak-arik (protein)

LUNCH:
  - Vegetable Beef Soup (Home Recipe) / Sup Daging Sapi Sayuran (Resep Rumahan) (protein)
  - Vegetable Soup (Home Recipe) / Sup Sayur (Resep Rumahan) (mixed)
  - Tomato Soup / Sup Tomat (mixed)
  - Chicken Soup / Sup Ayam (protein)
  - Mixed Beans Soup / Sup Kacang Campur (mixed)

DINNER:
  - Vegetable Beef Soup (Home Recipe) / Sup Daging Sapi Sayuran (Resep Rumahan) (protein)
  - Vegetable Soup (Home Recipe) / Sup Sayur (Resep Rumahan) (mixed)
  - Tomato Soup / Sup Tomat (mixed)
  - Chicken Soup / Sup Ayam (protein)
  - Mixed Beans Soup / Sup Kacang Campur (mixed)

SNACK:
  - Oil Popped White Popcorn / Popcorn Putih yang Dipanggang dengan Minyak (carbs)
  - Air Popped Popcorn / Popcorn yang Dipanggang dengan Udara (carbs)
  - Buttered Popcor

In [ ]:
# Check versatility (foods suitable for multiple meals)
df['meal_count'] = df[['suitable_breakfast', 'suitable_lunch', 'suitable_dinner', 'suitable_snack']].sum(axis=1)
print("\nFood versatility (suitable for multiple meals):")
print(df['meal_count'].value_counts().sort_index())
print(f"\nMost versatile foods (suitable for 3+ meals):")
versatile = df[df['meal_count'] >= 3][['food_name', 'food_name_id', 'category', 'meal_count']].head(10)
print(versatile.to_string(index=False))


Food versatility (suitable for multiple meals):
meal_count
0      3
1     23
2    167
3      7
Name: count, dtype: int64

Most versatile foods (suitable for 3+ meals):
          food_name             food_name_id category  meal_count
         Fried Rice              Nasi Goreng    carbs           3
Meatless Fried Rice Nasi Goreng Tanpa Daging    carbs           3
    Tempeh (Cooked)           Tempe (Matang)  protein           3
             Tempeh                    Tempe  protein           3
         Fried Tofu              Tahu Goreng  protein           3
   Soft Silken Tofu        Tahu Sutra Lembut  protein           3
   Firm Silken Tofu         Tahu Sutra Padat  protein           3


In [ ]:
# Drop temporary column
df = df.drop(columns=['meal_count'])

# Typical Serving Sizes
Assign realistic serving sizes based on Indonesian meal culture, all values in grams (g) or ml converted to grams


In [ ]:
def assign_serving_size(row):
    """
    Assign typical serving size in grams
    """
    category = row['category']
    food_name = str(row['food_name']).lower()

    # ==========================================
    # PRIORITY: Composite/Mixed Foods First
    # (Check these BEFORE category logic)
    # ==========================================

    # Sandwiches (prevent "cheese" from triggering dairy logic)
    if 'sandwich' in food_name:
        return 150  # Full sandwich

    # Soups (prevent individual ingredients from triggering)
    if 'soup' in food_name:
        return 300  # 1 bowl

    # Salads
    if 'salad' in food_name:
        return 300  # 1 bowl

    # Curries, Stews
    if any(kw in food_name for kw in ['curry', 'stew']):
        return 300  # 1 bowl

    # ==========================================
    # NOW Category-Based Logic
    # ==========================================

    # PROTEIN SOURCES
    if category == 'protein':
        if 'egg' in food_name:
            return 50
        elif any(kw in food_name for kw in ['tofu', 'tempeh']):
            return 100
        elif any(kw in food_name for kw in ['chicken', 'beef', 'pork', 'fish', 'salmon', 'tuna']):
            return 100
        elif any(kw in food_name for kw in ['shrimp', 'squid', 'crab', 'lobster']):
            return 100
        else:
            return 100

    # CARBOHYDRATES
    elif category == 'carbs':
        if 'rice' in food_name:
            return 150
        elif any(kw in food_name for kw in ['noodle', 'pasta', 'spaghetti']):
            return 150
        elif 'bread' in food_name:
            return 30
        elif 'potato' in food_name or 'potatoes' in food_name:
            return 150
        elif any(kw in food_name for kw in ['oatmeal', 'cereal']):
            return 40
        elif any(kw in food_name for kw in ['biscuit', 'cracker']):
            return 30
        elif 'popcorn' in food_name:
            return 25
        else:
            return 100

    # VEGETABLES
    elif category == 'vegetables':
        return 100  # Soups already handled above

    # FRUITS
    elif category == 'fruits':
        if 'banana' in food_name:
            return 120
        elif any(kw in food_name for kw in ['apple', 'orange', 'pear']):
            return 150
        elif any(kw in food_name for kw in ['mango', 'papaya', 'pineapple']):
            return 150
        elif any(kw in food_name for kw in ['grape', 'strawberry', 'blueberry']):
            return 100
        elif 'avocado' in food_name:
            return 100
        else:
            return 120

    # BEVERAGES
    elif category == 'beverages':
        return 240

    # DAIRY
    elif category == 'dairy':
        if 'yogurt' in food_name or 'yoghurt' in food_name:
            return 150
        elif 'cheese' in food_name:
            return 30  # Now only pure cheese items reach here!
        else:
            return 100

    # NUTS
    elif category == 'nuts':
        return 30

    # MIXED DISHES (fallback for any remaining)
    elif category == 'mixed':
        return 300  # Most mixed already handled in priority section

    # DEFAULT
    else:
        return 100

df['typical_serving_g'] = df.apply(assign_serving_size, axis=1)

In [ ]:
# Check for missing values
missing_servings = df['typical_serving_g'].isnull().sum()
print(f"\nMissing serving sizes: {missing_servings}")


Missing serving sizes: 0


In [ ]:
# Distribution of serving sizes
print("\nServing size distribution:")
print(df['typical_serving_g'].describe())


Serving size distribution:
count    200.000000
mean     115.100000
std       63.138119
min       25.000000
25%      100.000000
50%      100.000000
75%      150.000000
max      300.000000
Name: typical_serving_g, dtype: float64


In [ ]:
print("\nServing size ranges:")
print(f"  Very small (< 50g): {len(df[df['typical_serving_g'] < 50])} foods")
print(f"  Small (50-100g): {len(df[(df['typical_serving_g'] >= 50) & (df['typical_serving_g'] < 100)])} foods")
print(f"  Medium (100-150g): {len(df[(df['typical_serving_g'] >= 100) & (df['typical_serving_g'] < 150)])} foods")
print(f"  Large (150-250g): {len(df[(df['typical_serving_g'] >= 150) & (df['typical_serving_g'] < 250)])} foods")
print(f"  Very large (≥ 250g): {len(df[df['typical_serving_g'] >= 250])} foods")



Serving size ranges:
  Very small (< 50g): 24 foods
  Small (50-100g): 9 foods
  Medium (100-150g): 112 foods
  Large (150-250g): 40 foods
  Very large (≥ 250g): 15 foods


In [ ]:
# Serving sizes by category
print("\nAverage serving size by category:")
serving_by_category = df.groupby('category')['typical_serving_g'].agg(['mean', 'min', 'max', 'count'])
print(serving_by_category.round(1))


Average serving size by category:
             mean  min  max  count
category                          
carbs       102.6   25  150     38
dairy       130.9   30  300     11
fruits      130.6  100  150     17
mixed       281.2  150  300      8
nuts         30.0   30   30      7
protein     111.7   50  300     94
vegetables  100.0  100  100     25


In [ ]:
# Sample foods with serving sizes
print("\nSample foods with serving sizes (10 random items):")
sample_foods = df[['food_name', 'food_name_id', 'category', 'typical_serving_g', 'calories_per_100g']].sample(min(10, len(df)))
for idx, row in sample_foods.iterrows():
    # Calculate calories per serving
    cal_per_serving = (row['typical_serving_g'] / 100) * row['calories_per_100g']
    print(f"\n{row['food_name']} / {row['food_name_id']}")
    print(f"  Category: {row['category']}")
    print(f"  Serving: {row['typical_serving_g']}g")
    print(f"  Calories per serving: {cal_per_serving:.0f} kcal")


Sample foods with serving sizes (10 random items):

Coconut Meat / Daging Kelapa
  Category: fruits
  Serving: 120g
  Calories per serving: 425 kcal

Oatmeal / Oatmeal
  Category: carbs
  Serving: 40g
  Calories per serving: 25 kcal

Mackerel / Ikan Makarel
  Category: protein
  Serving: 100g
  Calories per serving: 167 kcal

Beef Fried Rice / Nasi Goreng Daging Sapi
  Category: protein
  Serving: 100g
  Calories per serving: 175 kcal

Red Kidney Beans (with Salt, Cooked, Boiled) / Kacang Merah (dengan Garam, Direbus, Dimasak)
  Category: vegetables
  Serving: 100g
  Calories per serving: 127 kcal

Bread / Roti
  Category: carbs
  Serving: 30g
  Calories per serving: 80 kcal

Squid (Mixed Species, Cooked, Fried) / Cumi-cumi (Campuran Spesies, Dimasak, Digoreng)
  Category: protein
  Serving: 100g
  Calories per serving: 175 kcal

Roasted Potato / Kentang Panggang
  Category: carbs
  Serving: 150g
  Calories per serving: 223 kcal

Chicken Drumstick / Paha Ayam (Paha Bawah)
  Category: 

In [ ]:
# Check extreme values
print("\nSmallest serving sizes (< 50g):")
small_servings = df[df['typical_serving_g'] < 50][['food_name', 'category', 'typical_serving_g']].head(5)
if len(small_servings) > 0:
    print(small_servings.to_string(index=False))
else:
    print("  None")

print("\nLargest serving sizes (≥ 300g):")
large_servings = df[df['typical_serving_g'] >= 300][['food_name', 'category', 'typical_serving_g']].head(5)
if len(large_servings) > 0:
    print(large_servings.to_string(index=False))
else:
    print("  None")


Smallest serving sizes (< 50g):
                       food_name category  typical_serving_g
        Oil Popped White Popcorn    carbs                 25
              Air Popped Popcorn    carbs                 25
  Buttered Popcorn Popped in Oil    carbs                 25
Unbuttered Popcorn Popped in Oil    carbs                 25
    Plain or Buttermilk Biscuits    carbs                 30

Largest serving sizes (≥ 300g):
                        food_name category  typical_serving_g
Vegetable Beef Soup (Home Recipe)  protein                300
     Vegetable Soup (Home Recipe)    mixed                300
                      Tomato Soup    mixed                300
                     Chicken Soup  protein                300
                 Mixed Beans Soup    mixed                300


In [ ]:
# Verify the fix
sandwiches = df[df['food_name'].str.contains('sandwich', case=False, na=False)]
print(f"\nSandwich servings after fix:")
print(sandwiches[['food_name', 'category', 'typical_serving_g']])



Sandwich servings after fix:
                                          food_name category  \
69                          Ham and Cheese Sandwich    dairy   
70                                         Sandwich    mixed   
71  Ham and Cheese Sandwich with Lettuce and Spread    dairy   
72                                 Chicken Sandwich  protein   
73                  Roast Beef Sandwich with Cheese  protein   
74                                  Cheese Sandwich    dairy   
75                                    Beef Sandwich  protein   

    typical_serving_g  
69                150  
70                150  
71                150  
72                150  
73                150  
74                150  
75                150  


# Derived Features (Ratios)

Add macro ratios for filtering/ranking logic

In [ ]:
# Calculate macro ratios (protein, carb, fat per calorie)

df['protein_ratio'] = df['protein_per_100g'] / df['calories_per_100g'].replace(0, 1)
df['carb_ratio'] = df['carbs_per_100g'] / df['calories_per_100g'].replace(0, 1)
df['fat_ratio'] = df['fat_per_100g'] / df['calories_per_100g'].replace(0, 1)

# Round to 4 decimal places for cleanliness
df['protein_ratio'] = df['protein_ratio'].round(4)
df['carb_ratio'] = df['carb_ratio'].round(4)
df['fat_ratio'] = df['fat_ratio'].round(4)

In [ ]:
# Check for missing values
print(f"\nMissing values:")
print(f"  protein_ratio: {df['protein_ratio'].isnull().sum()}")
print(f"  carb_ratio: {df['carb_ratio'].isnull().sum()}")
print(f"  fat_ratio: {df['fat_ratio'].isnull().sum()}")


Missing values:
  protein_ratio: 0
  carb_ratio: 0
  fat_ratio: 0


In [ ]:
# Distribution of ratios
print("\nProtein ratio distribution:")
print(df['protein_ratio'].describe().round(4))

print("\nCarb ratio distribution:")
print(df['carb_ratio'].describe().round(4))

print("\nFat ratio distribution:")
print(df['fat_ratio'].describe().round(4))


Protein ratio distribution:
count    200.0000
mean       0.0671
std        0.0461
min        0.0021
25%        0.0321
50%        0.0613
75%        0.0936
max        0.2096
Name: protein_ratio, dtype: float64

Carb ratio distribution:
count    200.0000
mean       0.1018
std        0.0874
min        0.0000
25%        0.0117
50%        0.0845
75%        0.1824
max        0.2666
Name: carb_ratio, dtype: float64

Fat ratio distribution:
count    200.0000
mean       0.0393
std        0.0271
min        0.0005
25%        0.0116
50%        0.0427
75%        0.0605
max        0.1037
Name: fat_ratio, dtype: float64


In [ ]:
# Check that ratios don't exceed 1.0 (impossible)
high_protein = df[df['protein_ratio'] > 1.0]
high_carb = df[df['carb_ratio'] > 1.0]
high_fat = df[df['fat_ratio'] > 1.0]

print(f"\nSanity checks:")
print(f"  Protein ratio > 1.0: {len(high_protein)} foods (should be 0)")
print(f"  Carb ratio > 1.0: {len(high_carb)} foods (should be 0)")
print(f"  Fat ratio > 1.0: {len(high_fat)} foods (should be 0)")


Sanity checks:
  Protein ratio > 1.0: 0 foods (should be 0)
  Carb ratio > 1.0: 0 foods (should be 0)
  Fat ratio > 1.0: 0 foods (should be 0)


In [ ]:
# Top foods by each macro ratio
print("\nTop 5 high-protein foods (by ratio):")
top_protein = df.nlargest(5, 'protein_ratio')[['food_name', 'food_name_id', 'category', 'protein_ratio', 'calories_per_100g']]
print(top_protein.to_string(index=False))

print("\nTop 5 high-carb foods (by ratio):")
top_carb = df.nlargest(5, 'carb_ratio')[['food_name', 'food_name_id', 'category', 'carb_ratio', 'calories_per_100g']]
print(top_carb.to_string(index=False))

print("\nTop 5 high-fat foods (by ratio):")
top_fat = df.nlargest(5, 'fat_ratio')[['food_name', 'food_name_id', 'category', 'fat_ratio', 'calories_per_100g']]
print(top_fat.to_string(index=False))


Top 5 high-protein foods (by ratio):
                food_name             food_name_id category  protein_ratio  calories_per_100g
                Egg White              Putih Telur  protein         0.2096          52.000000
Steamed or Boiled Lobster Lobster Rebus atau Kukus  protein         0.2092          97.029703
                     Crab                 Kepiting  protein         0.1980         100.990099
 Steamed or Boiled Shrimp   Udang Rebus atau Kukus  protein         0.1922         137.623762
                   Shrimp                    Udang  protein         0.1919         143.564356

Top 5 high-carb foods (by ratio):
                                                               food_name                                                        food_name_id category  carb_ratio  calories_per_100g
                                                                    Pear                                                                 Pir   fruits      0.2666               58.0
 

In [ ]:
# Sample foods with all ratios
print("\nSample foods with ratios (5 random):")
sample = df[['food_name', 'category', 'protein_ratio', 'carb_ratio', 'fat_ratio']].sample(5)
print(sample.to_string(index=False))


Sample foods with ratios (5 random):
                               food_name   category  protein_ratio  carb_ratio  fat_ratio
Chicken Soup with Dumplings and Potatoes    protein         0.0505      0.0980     0.0454
        Unbuttered Popcorn Popped in Oil      carbs         0.0138      0.1064     0.0610
                              Brown Rice      carbs         0.0233      0.2072     0.0081
                             Cauliflower vegetables         0.0792      0.2120     0.0040
                          Beef Meatballs    protein         0.0615      0.0375     0.0652


# Finalize Dataset

In [ ]:
df.columns

Index(['food_id', 'food_name', 'food_name_id', 'food_type', 'description',
       'search_keyword', 'serving_size', 'serving_unit', 'calories', 'protein',
       'carbs', 'fat', 'calories_per_100g', 'protein_per_100g',
       'carbs_per_100g', 'fat_per_100g', 'category', 'suitable_breakfast',
       'suitable_lunch', 'suitable_dinner', 'suitable_snack',
       'typical_serving_g', 'protein_ratio', 'carb_ratio', 'fat_ratio'],
      dtype='object')

In [ ]:
columns_to_drop = ['food_type', 'description']

df = df.drop(columns=columns_to_drop, errors='ignore')

In [ ]:
df.columns

Index(['food_id', 'food_name', 'food_name_id', 'search_keyword',
       'serving_size', 'serving_unit', 'calories', 'protein', 'carbs', 'fat',
       'calories_per_100g', 'protein_per_100g', 'carbs_per_100g',
       'fat_per_100g', 'category', 'suitable_breakfast', 'suitable_lunch',
       'suitable_dinner', 'suitable_snack', 'typical_serving_g',
       'protein_ratio', 'carb_ratio', 'fat_ratio'],
      dtype='object')

In [ ]:
column_order = [
    # Identifiers
    'food_id',
    'food_name',
    'food_name_id',
    'search_keyword',

    # Category & Classification
    'category',

    # Nutrition
    'calories',
    'protein',
    'carbs',
    'fat',

    # Nutrition (per 100g)
    'calories_per_100g',
    'protein_per_100g',
    'carbs_per_100g',
    'fat_per_100g',

    # Derived Features
    'protein_ratio',
    'carb_ratio',
    'fat_ratio',

    # Meal Suitability
    'suitable_breakfast',
    'suitable_lunch',
    'suitable_dinner',
    'suitable_snack',

    # Serving Size
    'serving_size'
    'typical_serving_g'
]

df = df[column_order].copy()

In [ ]:
# Export main dataset
output_filename = 'food_database_clean.csv'
df.to_csv(output_filename, index=False, encoding='utf-8')